In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Essential_Thrombocythemia/GSE174060'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data from CD34+ BMMCs/PBMCs from MPN patients vs. healthy controls"
!Series_summary	"Philadelphia chromosome-negative myeloproliferative neoplasms (MPN) consist of primary myelofibrosis (PMF), polycythemia vera (PV), essential thrombocythemia (ET) and seconday myelofibrosis (SMF), comprising post-ET-MF(pET-MF) and post-PV-MF(pPV-MF)."
!Series_summary	"In this dataset, we compare the gene expression data of bone marrow or peripheral blood mononuclear cells (BMMCs/PBMCs) of CD34+ cells from MPN patients and healthy donors."
!Series_summary	""
!Series_overall_design	"36 samples (6 ET, 11 PV, 9 PMF, 4 secondary MF (SMF) patients and 6 healthy donors) were analyzed"
Sample Characteristics Dictionary:
{0: ['tissue: bone marrow', 'tissue: peripheral blood'], 1: ['genotype: JAK2V617F', 'genotype: CALRins5', 'genotype: CALRdel52', 'genotype: MPLW515L', 'genotype: triple-negative', 'genotype: N/A'], 2: ['age: 41', 'age: 53', 'age: 52', 'age:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is likely present based on the background info
is_gene_available = True  # Based on the series title and summary

# Identify keys for the required variables
trait_row = 4  # Corresponding to 'diagnosis: ET'
age_row = 2  # Corresponding to 'age'
gender_row = 3  # Corresponding to 'Sex'

# Define conversion functions
def convert_trait(value):
    try:
        diagnosis = value.split(': ')[1]
        return 1 if diagnosis == 'ET' else 0
    except:
        return None

def convert_age(value):
    try:
        age = int(value.split(': ')[1])
        return age  # Age is continuous
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(': ')[1]
        return 1 if gender == 'M' else 0 if gender == 'F' else None
    except:
        return None

# Save cohort information
save_cohort_info('GSE174060', './preprocessed/Essential_Thrombocythemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Essential_Thrombocythemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Essential_Thrombocythemia/trait_data/GSE174060.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM5285411': [1, 41, 0], 'GSM5285412': [1, 53, 0], 'GSM5285413': [1, 52, 1], 'GSM5285414': [1, 47, 1], 'GSM5285415': [1, 19, 1], 'GSM5285416': [1, 33, 0], 'GSM5285417': [0, 58, 1], 'GSM5285418': [0, 76, 1], 'GSM5285419': [0, 68, 1], 'GSM5285420': [0, 65, 0], 'GSM5285421': [0, 61, 1], 'GSM5285422': [0, 45, 1], 'GSM5285423': [0, 68, 0], 'GSM5285424': [0, 42, 1], 'GSM5285425': [0, 36, 0], 'GSM5285426': [0, 42, 1], 'GSM5285427': [0, 69, 0], 'GSM5285428': [0, 45, 1], 'GSM5285429': [0, 74, 1], 'GSM5285430': [0, 75, 1], 'GSM5285431': [0, 62, 1], 'GSM5285432': [0, 58, 1], 'GSM5285433': [0, 76, 1], 'GSM5285434': [0, 72, 0], 'GSM5285435': [0, 61, 0], 'GSM5285436': [0, 76, 1], 'GSM5285437': [0, 71, 0], 'GSM5285438': [0, 43, 0], 'GSM5285439': [0, 56, 0], 'GSM5285440': [0, 68, 0], 'GSM5285441': [0, 29, 0], 'GSM5285442': [0, 27, 0], 'GSM5285443': [0, 28, 1], 'GSM5285444': [0, 28, 1], 'GSM5285445': [0, 32, 0], 'GSM5285446': [0, 27, 0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1',
       'TC01000004.hg.1', 'TC01000005.hg.1', 'TC01000006.hg.1',
       'TC01000007.hg.1', 'TC01000008.hg.1', 'TC01000009.hg.1',
       'TC01000010.hg.1', 'TC01000011.hg.1', 'TC01000012.hg.1',
       'TC01000013.hg.1', 'TC01000014.hg.1', 'TC01000015.hg.1',
       'TC01000016.hg.1', 'TC01000017.hg.1', 'TC01000018.hg.1',
       'TC01000019.hg.1', 'TC01000020.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the key names for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
# Further parsing the gene_assignment field to correctly map entries
# Extract first gene symbol assignment

def parse_gene_assignment(assignment):
    try:
        return assignment.split("//")[1].split("//")[0].strip()
    except:
        return None

gene_annotation[gene_symbol_key] = gene_annotation[gene_symbol_key].apply(parse_gene_assignment)
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Printing the preview of resulting gene_data for verification, if needed
print(preview_df(gene_data))


{'GSM5285411': [5.595281665151299, 5.427588891, 5.452647829, 3.828432838, 10.55324276], 'GSM5285412': [5.648892395687045, 5.283054009, 5.478249536, 3.652119479, 7.151121683], 'GSM5285413': [5.68491207945691, 5.488521992, 5.324153595, 3.851147778, 11.29035778], 'GSM5285414': [5.652017337313328, 5.407397385, 5.423946373, 4.110067827, 11.16524002], 'GSM5285415': [5.587275358246945, 5.265102337, 5.540004137, 3.912271785, 8.716703861], 'GSM5285416': [5.626453181197286, 5.212153116, 5.190079949, 4.01225781, 8.34763814], 'GSM5285417': [5.624870796111825, 5.263750295, 5.479629816, 3.879794314, 6.851918907], 'GSM5285418': [5.612551726525101, 5.360106883, 5.236413605, 4.06967024, 7.549544858], 'GSM5285419': [5.662011242854438, 5.273375632, 5.509132862, 4.234829069, 6.953907771], 'GSM5285420': [5.546538122624075, 5.315716674, 5.422619716, 4.010396216, 6.899888675], 'GSM5285421': [5.494424929928596, 5.122157622, 5.197685537, 3.775524627, 9.499557664], 'GSM5285422': [5.481274826476533, 5.360463381,

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Essential_Thrombocythemia/gene_data/GSE174060.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Essential_Thrombocythemia')

# Define trait_biased based on trait_biased
trait_biased = trait_biased

# 4. Save the cohort information.
save_cohort_info('GSE174060', './preprocessed/Essential_Thrombocythemia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Essential_Thrombocythemia/GSE174060.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 6 occurrences. This represents 16.67% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

Quartiles for 'Age':
  25%: 39.75
  50% (Median): 54.5
  75%: 68.0
Min: 19.0
Max: 76.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 17 occurrences. This represents 47.22% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

